#In-Class Assignment #2
Coke Data

Data on 1,140 customers who purchased Coke or Pepsi

Price of Pepsi, Price of Coke, Advertising displays

Binary Choice: coke = 1 if Coke is chosen, = 0 if Pepsi is chosen

Implement Logistic Regression (y = coke, X = constant, pratio, disp_pepsi, disp_coke) and explain (discuss) the estimation result applying
(1) an Econometric Approach (using all data) and
(2) a Machine Learning Approach (using 80% training data and 20% test data) applying a sklearn pipeline.

In [11]:
import pandas as pd
import numpy as np

# Econometrics
import statsmodels.api as sm

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [3]:
coke = pd.read_csv('http://www.principlesofeconometrics.com/poe5/data/csv/coke.csv')

In [9]:
coke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   coke        1140 non-null   int64  
 1   pr_pepsi    1140 non-null   float64
 2   pr_coke     1140 non-null   float64
 3   disp_pepsi  1140 non-null   int64  
 4   disp_coke   1140 non-null   int64  
 5   pratio      1140 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 53.6 KB


In [4]:
coke

,coke,pr_pepsi,pr_coke,disp_pepsi,disp_coke,pratio
0,1,1.79,1.79,0,0,1.000000
1,1,1.79,0.89,0,1,0.497207
2,1,1.41,0.89,0,0,0.631206
3,1,1.79,1.33,0,0,0.743017
4,1,1.79,1.79,0,0,1.000000
...,...,...,...,...,...,...
1135,1,0.99,1.79,0,0,1.808081
1136,1,1.79,0.89,0,1,0.497207
1137,1,1.09,1.33,0,1,1.220183
1138,1,0.89,0.89,1,1,1.000000


In [5]:
coke.isnull().sum()

,0
coke,0
pr_pepsi,0
pr_coke,0
disp_pepsi,0
disp_coke,0
pratio,0


In [8]:
coke.duplicated().sum()
#shoulbe be considered

np.int64(926)

In [10]:
coke.corr()

,coke,pr_pepsi,pr_coke,disp_pepsi,disp_coke,pratio
coke,1.000000,0.269817,-0.102307,-0.233404,0.111776,-0.321375
pr_pepsi,0.269817,1.000000,0.346991,-0.627470,-0.281487,-0.527249
pr_coke,-0.102307,0.346991,1.000000,-0.157059,-0.657734,0.581570
disp_pepsi,-0.233404,-0.627470,-0.157059,1.000000,0.265827,0.403133
disp_coke,0.111776,-0.281487,-0.657734,0.265827,1.000000,-0.342457
pratio,-0.321375,-0.527249,0.581570,0.403133,-0.342457,1.000000


#**ECONOMETRIC APPROACH**

prepare for x,y

In [12]:
y = coke['coke']

X = coke[['pratio', 'disp_pepsi', 'disp_coke']]
X = sm.add_constant(X)   # thêm constant


Logistic Regression

In [13]:
logit_model = sm.Logit(y, X)
result = logit_model.fit()
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.622321
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   coke   No. Observations:                 1140
Model:                          Logit   Df Residuals:                     1136
Method:                           MLE   Df Model:                            3
Date:                Thu, 08 Jan 2026   Pseudo R-squ.:                 0.09493
Time:                        01:45:19   Log-Likelihood:                -709.45
converged:                       True   LL-Null:                       -783.86
Covariance Type:            nonrobust   LLR p-value:                 4.715e-32
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9230      0.326      5.902      0.000       1.284       2.562
pratio        -1.9957      0.

# **Logistic Regression (Logit) to predict:**

The likelihood of a customer choosing Coke (coke = 1) compared to choosing Pepsi (coke = 0)




The logistic regression results show that price and advertising displays significantly influence consumers’ choices between Coke and Pepsi, as all explanatory variables are statistically significant at the 5% level.

The price ratio has a strong negative effect on the probability of choosing Coke, with an estimated coefficient of −1.996 (p < 0.01). This implies that a one-unit increase in the price ratio reduces the odds of choosing Coke by approximately 86% (e⁻¹·⁹⁹⁶ ≈ 0.14), indicating that consumers are highly price-sensitive.

Advertising displays also play an important role. The coefficient on Pepsi’s display is −0.731 (p < 0.01), suggesting that when Pepsi is promoted, the odds of choosing Coke decrease by about 52% (e⁻⁰·⁷³¹ ≈ 0.48). In contrast, Coke’s display has a positive and statistically significant coefficient of 0.352 (p = 0.027), which increases the odds of choosing Coke by approximately 42% (e⁰·³⁵² ≈ 1.42).

Overall, these results suggest that pricing strategy is more influential than advertising in determining consumer choice.

- exp(beta) > 1 → tăng odds chọn Coke
- exp(beta) < 1 → giảm odds chọn Coke


In [14]:
odds_ratio = np.exp(result.params)
odds_ratio


,0
const,6.841262
pratio,0.135913
disp_pepsi,0.481434
disp_coke,1.421339


#**MACHINE LEARNING APPROACH (80% Train – 20% Test)**

In [15]:
X_ml = coke[['pratio', 'disp_pepsi', 'disp_coke']]
y_ml = coke['coke']


Split the train and test

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X_ml, y_ml,
    test_size=0.2,
    random_state=42
)


Pipeline (Scaler + Logistic Regression)

In [17]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logit', LogisticRegression())
])


Train model

In [18]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('logit', LogisticRegression())])

Predict & Evaluate

In [19]:
y_pred = pipe.predict(X_test)

# Accuracy
accuracy_score(y_test, y_pred)


0.6842105263157895

In [20]:
# Confusion Matrix
confusion_matrix(y_test, y_pred)

array([[112,  26],
       [ 46,  44]])

In [21]:
# Classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.71      0.81      0.76       138
           1       0.63      0.49      0.55        90

    accuracy                           0.68       228
   macro avg       0.67      0.65      0.65       228
weighted avg       0.68      0.68      0.68       228



The classification report shows that the logistic regression model achieves an overall accuracy of 68% on the test data. The model performs better in predicting Pepsi choices than Coke choices, with a recall of 0.81 for class 0 compared to 0.49 for class 1. This indicates that while the model correctly identifies most Pepsi consumers, it fails to capture a substantial proportion of Coke consumers. Overall, the predictive performance is moderate, suggesting reasonable generalization but leaving room for improvement.

# Optional to compare the (hệ số)

In [22]:
coef = pipe.named_steps['logit'].coef_
coef


array([[-0.54185992, -0.31880713,  0.15338294]])